In [2]:
from dotenv import load_dotenv
import os
import openai
import llama_index
from llama_index import Document
from llama_index import StorageContext, ServiceContext
from llama_index.storage.storage_context import StorageContext
from llama_index.embeddings.openai import OpenAIEmbedding
from pathlib import Path
from llama_index.embeddings import OpenAIEmbedding
from llama_index.text_splitter import SentenceSplitter
from llama_index.ingestion import IngestionPipeline, IngestionCache
from pathlib import Path
from llama_index import download_loader
from llama_index.indices.document_summary import DocumentSummaryIndex
from llama_index.llms import OpenAI
from llama_index import (
    SimpleDirectoryReader,
    get_response_synthesizer,
)
from llama_index.ingestion import IngestionPipeline
import asyncio
from llama_index import (
    load_index_from_storage,
    set_global_service_context
)
from llama_index.callbacks import CallbackManager, TokenCountingHandler
from llama_index.node_parser import SentenceSplitter
import tiktoken
from llama_index import set_global_handler

In [176]:
set_global_handler("simple")

In [3]:
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [142]:
storage_context = StorageContext.from_defaults(
    docstore = SimpleDocumentStore.from_persist_dir(persist_dir="C:/Users/jrdef/Documents/ds384"),
    index_store = SimpleIndexStore.from_persist_dir(persist_dir="C:/Users/jrdef/Documents/ds384")
)

ds384 = load_index_from_storage(storage_context)

In [4]:
PDFReader = download_loader("PDFReader")

loader = PDFReader()
documents = loader.load_data(file=Path('C:/Users/jrdef/Downloads/psych.pdf'))

In [5]:
documents

[Document(id_='069d68ce-a0fa-47e9-969b-449579a91116', embedding=None, metadata={'page_label': '1', 'file_name': 'psych.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='c8e35a809e0f86d8852f77ccefa25316988abd49a93e739c88e024ed4d6fcb0f', text='404Is Intelligence One General\nAbility or Several Specific\nAbilities?\nIntelligence and Creativity\nEmotional Intelligence\nIs Intelligence Neurologically\nMeasurable?module 33\nIntroduction to Intelligence\nSchool boards, courts, and scientists debate the use and fairness of tests that attempt\nto assess people’s mental abilities and assign them a score. Is intelligence testing aconstructive way to guide people toward suitable opportunities? Or is it a potent, dis-criminatory weapon camouflaged as science? First, some basic questions: \n/H17012What is intelligence? \n/H17012How can we best assess it?\n/H17012To what extent does it result from heredity rather than environment? \n/H17012What do test sco

In [6]:
combined_text = ''

for document in documents:
    # Assuming each document has a .text attribute
    combined_text += document.text + " "

In [7]:
combined_text

'404Is Intelligence One General\nAbility or Several Specific\nAbilities?\nIntelligence and Creativity\nEmotional Intelligence\nIs Intelligence Neurologically\nMeasurable?module 33\nIntroduction to Intelligence\nSchool boards, courts, and scientists debate the use and fairness of tests that attempt\nto assess people’s mental abilities and assign them a score. Is intelligence testing aconstructive way to guide people toward suitable opportunities? Or is it a potent, dis-criminatory weapon camouflaged as science? First, some basic questions: \n/H17012What is intelligence? \n/H17012How can we best assess it?\n/H17012To what extent does it result from heredity rather than environment? \n/H17012What do test score differences among individuals and groups really mean?Should we use such differences to rank people, to admit them to colleges oruniversities, to hire them?\nPsychologists debate: Should we consider intelligence as one aptitude or many? As\nlinked to cognitive speed? As neurologicall

In [6]:
psych = documents[1:3]

In [39]:
len(documents)

4

In [37]:
documents[1].text

'405 Introduction to Intelligence MODULE 33\n/H17012||Is Intelligence One General Ability \nor Several Specific Abilities?\nWhat arguments support intelligence as one general mental ability, and\nwhat arguments support the idea of multiple distinct abilities?\nYou probably know some people with talents in science, others who excel at the hu-\nmanities, and still others gifted in athletics, art, music, or dance. You may also knowa talented artist who is dumbfounded by the simplest mathematical problems, or abrilliant math student with little aptitude for literary discussion. Are all of these peo-ple intelligent? Could you rate their intelligence on a single scale? Or would you needseveral different scales? \nCharles Spearman (1863–1945) believed we have one general intelligence (often\nshortened to g). He granted that people often have special abilities that stand out.\nSpearman had helped develop factor analysis, a statistical procedure that identifies\nclusters of related items. He ha

In [125]:
(512+1024) / 2

768.0

In [8]:
sentence_parser = SentenceSplitter(chunk_size= 768, chunk_overlap= 200)

text_chunks = sentence_parser.split_text(combined_text)
doc_chunks = [Document(text=t) for t in text_chunks]

In [9]:
len(doc_chunks[5].text.split())

85

In [10]:
doc_chunks

[Document(id_='ae893270-6b45-4594-9b73-1ac9aeaab318', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='a18528225bdc1c7052b9c280b26e592efc5e7973cea7c77c53c46300aa137140', text='404Is Intelligence One General\nAbility or Several Specific\nAbilities?\nIntelligence and Creativity\nEmotional Intelligence\nIs Intelligence Neurologically\nMeasurable?module 33\nIntroduction to Intelligence\nSchool boards, courts, and scientists debate the use and fairness of tests that attempt\nto assess people’s mental abilities and assign them a score. Is intelligence testing aconstructive way to guide people toward suitable opportunities? Or is it a potent, dis-criminatory weapon camouflaged as science? First, some basic questions: \n/H17012What is intelligence? \n/H17012How can we best assess it?\n/H17012To what extent does it result from heredity rather than environment? \n/H17012What do test score differences among individuals and groups

index entire document first, then document split action, then run for loop query on top of the documents

In [7]:
psych

[Document(id_='3c16660c-0ed1-4781-b856-357ebaf93d83', embedding=None, metadata={'page_label': '2', 'file_name': 'psych.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='97030634c0b37911b0a60293fe56ebbc3f3fdeb92ff8ecf685471723ef35f20b', text='405 Introduction to Intelligence MODULE 33\n/H17012||Is Intelligence One General Ability \nor Several Specific Abilities?\nWhat arguments support intelligence as one general mental ability, and\nwhat arguments support the idea of multiple distinct abilities?\nYou probably know some people with talents in science, others who excel at the hu-\nmanities, and still others gifted in athletics, art, music, or dance. You may also knowa talented artist who is dumbfounded by the simplest mathematical problems, or abrilliant math student with little aptitude for literary discussion. Are all of these peo-ple intelligent? Could you rate their intelligence on a single scale? Or would you needseveral different scales?

## INITIALIZE TOKENS

In [11]:
token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode
)

callback_manager = CallbackManager([token_counter])

## PROMPTS

In [20]:
define_prompt = """

Based on the provided summary, your task is to identify and list all terms along with their definitions. 

Each term should be followed by its corresponding definition. 

I a human name is present, if it is relevant, it should be counted as a term and followed by a definition.

If a term's definition is not explicitly stated, use your best judgment to infer it based on the context.

Format your response as follows: 
Term: <term> Definition: <definition>

Focus on capturing a wide range of terms, prioritizing the quantity of terms identified. 

Precision in the definitions is important but secondary to the relevance of terms extracted.

To assist you in understanding the context and relevance, here is the summary of the text: '{}'

"""

summaryPrompt3 = (
    """Task: Conduct a detailed analysis of this '{}' in these specific steps:
    1. Meticulously identify and list every human name mentioned, focusing on both historical figures and any other individuals. Pay special attention to ensure no name is overlooked.
    2. Carefully extract and enumerate all events detailed in the text, including both major and minor occurrences, ensuring a comprehensive capture of the timeline.
    3. Thoroughly identify all significant terms and concepts, including technical jargon, theories, or specific subject matter terminology.
    4. Make sure to get as many terms as possible in this summary 
    After completing these steps, synthesize this information into a cohesive and comprehensive summary. The summary should seamlessly incorporate the identified names, events, and terms, providing a clear, accurate, and complete overview of the text's content."""
)

## OBSERVABILITY

In [11]:
llama_index.set_global_handler("simple")

## BUILD LLM MODEL AND INDEXING

In [12]:
def get_llm(openai_api_key, max_tokens=8192):

    os.environ["OPENAI_API_KEY"] = openai_api_key
    return OpenAI(
        temperature=0.0, model='gpt-3.5-turbo-16k', max_tokens=max_tokens
    )

In [13]:
llm = get_llm(OPENAI_API_KEY)

In [14]:
def get_ds_index(documents, llm, c_m, api_key):

    service_context = ServiceContext.from_defaults(
        llm=llm, 
        chunk_size=384,
        callback_manager = c_m, 
        chunk_overlap=128
        )
    
    set_global_service_context(service_context)
    
    response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=False
    )
        
    temp_index = DocumentSummaryIndex.from_documents(
        documents, 
        service_context=service_context,
        response_synthesizer= response_synthesizer
    )

    return temp_index

In [15]:
docs_index = get_ds_index(doc_chunks, llm, callback_manager, OPENAI_API_KEY)

current doc id: ae893270-6b45-4594-9b73-1ac9aeaab318
current doc id: 2f7121b0-6ede-4f40-98b9-50dc81f07b98
current doc id: fcc19f0b-3a01-40f8-9765-517e3f37bbc5
current doc id: 5521f165-e892-41af-b5e8-8eba4ead9561
current doc id: bad2e389-8514-4f14-b20c-0e360538bb1b
current doc id: 28355251-85d3-4f59-8926-476dfc1c1012


In [16]:
print(token_counter.total_embedding_token_count)

1055


In [17]:
def summarize_docs(index, prompt, k=3):
    query_engine = index.as_query_engine(similarity_top_k=k)

    summary = str(query_engine.query(prompt))

    return summary


In [92]:
psych

[Document(id_='3c16660c-0ed1-4781-b856-357ebaf93d83', embedding=None, metadata={'page_label': '2', 'file_name': 'psych.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='97030634c0b37911b0a60293fe56ebbc3f3fdeb92ff8ecf685471723ef35f20b', text='405 Introduction to Intelligence MODULE 33\n/H17012||Is Intelligence One General Ability \nor Several Specific Abilities?\nWhat arguments support intelligence as one general mental ability, and\nwhat arguments support the idea of multiple distinct abilities?\nYou probably know some people with talents in science, others who excel at the hu-\nmanities, and still others gifted in athletics, art, music, or dance. You may also knowa talented artist who is dumbfounded by the simplest mathematical problems, or abrilliant math student with little aptitude for literary discussion. Are all of these peo-ple intelligent? Could you rate their intelligence on a single scale? Or would you needseveral different scales?

In [18]:
def prepare_docs_for_summary(index, prompt, documents, k=3):
    query_engine = index.as_query_engine(similarity_top_k=k)
    formatted_prompts = []
    for document in documents:
        formatted_prompts.append(prompt.format(document.text))
    return query_engine, formatted_prompts
       
async def summarize_many_docs(query_engine, prompts):
    tasks = [query_engine.aquery(p) for p in prompts]
    r = await asyncio.gather(*tasks)
    return r
    



In [22]:
q_e, f_p = prepare_docs_for_summary(docs_index, summaryPrompt3, doc_chunks)

In [23]:
summaries = await (summarize_many_docs(q_e, f_p))


In [24]:
summaries

[Response(response="The context discusses the debate surrounding intelligence testing and its use in assessing people's mental abilities. It raises questions about the nature of intelligence, how it should be assessed, and the extent to which it is influenced by heredity or environment. Psychologists debate whether intelligence should be considered as one general aptitude or as multiple distinct abilities. The concept of intelligence is described as socially constructed, with different cultures defining intelligence based on the attributes that enable success in their specific contexts. Intelligence is defined as the ability to learn from experience, solve problems, and adapt to new situations. Intelligence tests are used to measure intelligence, but it is emphasized that intelligence is not a fixed trait like height, but rather a concept that is measured through test scores. The context also mentions the idea of multiple intelligences, as proposed by Howard Gardner, which suggests tha

In [25]:
ttl_summary = " ".join([r.response for r in summaries])

In [26]:
ttl_summary

'The context discusses the debate surrounding intelligence testing and its use in assessing people\'s mental abilities. It raises questions about the nature of intelligence, how it should be assessed, and the extent to which it is influenced by heredity or environment. Psychologists debate whether intelligence should be considered as one general aptitude or as multiple distinct abilities. The concept of intelligence is described as socially constructed, with different cultures defining intelligence based on the attributes that enable success in their specific contexts. Intelligence is defined as the ability to learn from experience, solve problems, and adapt to new situations. Intelligence tests are used to measure intelligence, but it is emphasized that intelligence is not a fixed trait like height, but rather a concept that is measured through test scores. The context also mentions the idea of multiple intelligences, as proposed by Howard Gardner, which suggests that intelligence is 

In [18]:
def extract_terms_1(index, prompt, summary, k=3):
    query_engine = index.as_query_engine(
        similarity_top_k=k
    )

    f_prompt = prompt.format(summary)

    terms_definitions = str(query_engine.query(f_prompt))

    terms_definitions = [
        x
        for x in terms_definitions.split("\n")
        if x and "Term:" in x and "Definition:" in x
    ]
    # parse the text into a dict
    terms_to_definition = {
        x.split("Definition:")[0]
        .split("Term:")[-1]
        .strip(): x.split("Definition:")[-1]
        .strip()
        for x in terms_definitions
    }
    return terms_to_definition, query_engine

In [28]:
def extract_terms_2(index, prompt, summary, k=3):
    query_engine = index.as_query_engine(
        similarity_top_k=k
    )

    f_prompt = prompt.format(summary)

    terms_definitions = str(query_engine.query(f_prompt))

    terms_definitions = [
        x
        for x in terms_definitions.split("\n")
        if x and "Term:" in x and "Definition:" in x
    ]
    # parse the text into a dict
    terms_to_definition = {
        x.split("Definition:")[0]
        .split("Term:")[-1]
        .strip(): x.split("Definition:")[-1]
        .strip()
        for x in terms_definitions
    }

    return terms_to_definition, query_engine

In [27]:
print(ttl_summary)

The context discusses the debate surrounding intelligence testing and its use in assessing people's mental abilities. It raises questions about the nature of intelligence, how it should be assessed, and the extent to which it is influenced by heredity or environment. Psychologists debate whether intelligence should be considered as one general aptitude or as multiple distinct abilities. The concept of intelligence is described as socially constructed, with different cultures defining intelligence based on the attributes that enable success in their specific contexts. Intelligence is defined as the ability to learn from experience, solve problems, and adapt to new situations. Intelligence tests are used to measure intelligence, but it is emphasized that intelligence is not a fixed trait like height, but rather a concept that is measured through test scores. The context also mentions the idea of multiple intelligences, as proposed by Howard Gardner, which suggests that intelligence is co

In [29]:
d3, qe1 = extract_terms_2(docs_index, define_prompt, ttl_summary)

In [30]:
d3

{}

In [19]:
token_counter.reset_counts()

In [81]:
def calculate_and_print_costs(token_counter):
    # Calculate the costs
    embedding_token_cost = (token_counter.total_embedding_token_count / 1000) * 0.0001
    llm_prompt_token_cost = (token_counter.prompt_llm_token_count / 1000) * 0.001
    llm_completion_token_cost = (token_counter.completion_llm_token_count / 1000) * 0.002
    total_cost = embedding_token_cost + llm_prompt_token_cost + llm_completion_token_cost

    # Print the details
    print("Embedding Tokens: ", token_counter.total_embedding_token_count,". $", embedding_token_cost)
    print("LLM Prompt Tokens: ", token_counter.prompt_llm_token_count, ". $", llm_prompt_token_cost)
    print("LLM Completion Tokens: ", token_counter.completion_llm_token_count, ". $", llm_completion_token_cost)
    print("Total LLM Token Count: ", token_counter.total_llm_token_count, ". $", total_cost, "\n")

In [82]:
calculate_and_print_costs(token_counter)

Embedding Tokens:  0 . $ 0.0
LLM Prompt Tokens:  0 . $ 0.0
LLM Completion Tokens:  0 . $ 0.0
Total LLM Token Count:  0 . $ 0.0 



In [1]:
def do_2(index, summarize_prompt, ext_prompt):
    ## do this by the chunks
    summary = summarize_docs(index, summarize_prompt, 3)
    print("Summary of Documents")
    
    terms, extract_engine = extract_terms_1(index, ext_prompt, summary, 3)
    print("Initial Terms / Definitions")

    calculate_and_print_costs(token_counter)
    
    return summary, terms

In [26]:
s1, d1 = do_2(index_psych, summaryPrompt3, define_prompt)

Summary of Documents
Initial Terms / Definitions
Embedding Tokens:  1308 . $ 0.0001308
LLM Prompt Tokens:  10634 . $ 0.010634000000000001
LLM Completion Tokens:  1201 . $ 0.002402
Total LLM Token Count:  11835 . $ 0.013166800000000001 



In [62]:
d2 = extract_terms_1(index_psych, define_prompt, s1)

In [63]:
d2

({'Savant syndrome': 'A condition in which a person with limited mental ability has an exceptional specific skill.',
  "Howard Gardner's theory of multiple intelligences": 'The idea that intelligence is composed of multiple abilities that come in packages.',
  'General intelligence (g)': 'A common skill set that underlies all intelligent behavior.',
  'Factor analysis': 'A statistical procedure used to identify clusters of related items.',
  "L. L. Thurstone's theory of primary mental abilities": 'The idea that there are seven clusters of primary mental abilities.',
  "Satoshi Kanazawa's theory of general intelligence": 'The idea that general intelligence evolved as a form of intelligence that helps people solve novel problems.',
  "Robert Plomin's view on general intelligence": 'The belief that general intelligence is one of the most reliable and valid measures in the behavioral domain.'},
 <llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x24bf5c59110>)

In [30]:
d1

({'Savant syndrome': 'A condition in which a person with limited mental ability has an exceptional specific skill.',
  "Howard Gardner's theory of multiple intelligences": 'The idea that intelligence is composed of multiple abilities that come in packages.',
  'General intelligence (g)': 'A common skill set that underlies all intelligent behavior.',
  'Factor analysis': 'A statistical procedure used to identify clusters of related items.',
  "L. L. Thurstone's theory of primary mental abilities": 'The idea that there are seven clusters of primary mental abilities.',
  "Satoshi Kanazawa's theory of general intelligence": 'The idea that general intelligence evolved as a form of intelligence that helps people solve novel problems.',
  "Robert Plomin's view on general intelligence": 'The belief that general intelligence is one of the most reliable and valid measures in the behavioral domain.'},
 <llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x24bf2f04450>)

## EVALUATOR ( NOT COMPLETELY SURE 'HOW' THIS EVALS IF ITS ON THE INDEX OR SIMPLY ON THE RAG DATASET)

In [227]:
RagEvaluatorPack = download_llama_pack("RagEvaluatorPack", "./pack")

async def eval(query_engine, rag_dataset):
    rag_evaluator = RagEvaluatorPack(
    query_engine=query_engine,
    rag_dataset=rag_dataset,  # defined in 1A
    show_progress=True,
    )

   
    benchmark_df =  await rag_evaluator.arun(
        batch_size = 5,
        sleep_time_in_seconds = 15
    )

    print(benchmark_df)
    return benchmark_df

In [224]:
bm = await eval(b1, rag_dataset)

Batch processing of predictions: 100%|██████████| 2/2 [00:26<00:00, 13.47s/it]
Batch processing of evaluations:  67%|██████▋   | 2/3.0 [01:06<00:33, 33.08s/it]

rag                            base_rag
metrics                                
mean_correctness_score         5.000000
mean_relevancy_score           1.000000
mean_faithfulness_score        1.000000
mean_context_similarity_score  0.787991
